# Testing new intake callback

installed manually with `pip install git+https://github.com/NCAR/intake-esm.git`

In [1]:
import intake

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
col = intake.open_esm_datastore("../cmip6hack-ocean-bgc/catalogs/pangeo-cmip6.json")

In [3]:
# dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False}, 
#                                 cdf_kwargs={'chunks': {}, 'decode_times': False})

In [11]:
query = dict(experiment_id='historical', table_id='Oyr', 
                 variable_id='o2', grid_label='gn', member_id='r1i1p1f1', source_id='CESM2-WACCM')
cat = col.search(**query)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
13436,CMIP,NCAR,CESM2-WACCM,historical,r1i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/NCAR/CESM2-WACCM/historical/r1...,NaN


In [12]:
from cmip6_preprocessing.preprocessing import rename_cmip6

In [13]:
dsets = cat.to_dataset_dict()

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)


In [24]:
def test_func(ds):
    if 'nlon' in ds.dims:
        ds = ds.rename({'nlon':'WAZOOOOOONGA'})
    return ds

In [25]:
# dsets_pp = cat.to_dataset_dict(preprocess=rename_cmip6)
dsets_pp = cat.to_dataset_dict(preprocess=test_func)

In [26]:
dsets_pp['CMIP.NCAR.CESM2-WACCM.historical.Oyr.gn'].dims

Frozen(SortedKeysDict({'nlat': 384, 'nlon': 320, 'vertices': 4, 'time': 165, 'd2': 2, 'lev': 60, 'member_id': 1}))

In [27]:
test_func(dsets_pp['CMIP.NCAR.CESM2-WACCM.historical.Oyr.gn']).dims

Frozen(SortedKeysDict({'d2': 2, 'lev': 60, 'member_id': 1, 'nlat': 384, 'WAZOOOOOONGA': 320, 'time': 165, 'vertices': 4}))